In [1]:
import torch
import torch.nn as nn
from torchvision import models

from Data.loader_from_array import MedMNIST_Testing
from Train.train_one_epoch import eval_epoch_save

In [2]:
model = models.densenet121()
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 2))

In [3]:
names = ['features.denseblock1.', 'features.denseblock2.', 'features.denseblock3.', 'features.denseblock4.', 'classifier.']

In [4]:
loaded = torch.load('/home/zoe/GhassanGT Dropbox/Zoe Fowler/Zoe/InSync/PhDResearch/Code/Federated-SuperResolution/Saved/pneumonia/classification/model.pth')
model.load_state_dict(loaded)

<All keys matched successfully>

In [5]:
model = model.cuda()


In [22]:
import numpy as np


def get_gradients(data_loader, model, criterion, device='cuda'):
    
    total_grads = []
        
    for i, (image, target) in enumerate(data_loader):
        
        layer_grads = {}
        for pref in names:
            layer_grads[pref] = []
        #layer_grads = []
            
        one_hot = torch.zeros(target.shape[0], 2)
        one_hot[range(target.shape[0]), target.long()] = 1
        input = image.to(device)
        one_hot = one_hot.to(device)
        input = input.float()
        model.zero_grad()
        output = model(input)
        loss = criterion(output, one_hot)
        # backward pass (compute gradients of parameters w.r.t. loss)
        loss.backward()

        for name, param in model.named_parameters():
            if name in names:
                if param.grad is not None:
                    #layer_grads.append(np.mean(np.linalg.norm(param.grad.detach().cpu().numpy())))
                    layer_grads[pref].append(np.linalg.norm(param.grad.detach().cpu().numpy()))

        # for pref in names:
        #     if len(layer_grads[pref]) > 0:
        #         layer_grads[pref] = np.mean(layer_grads[pref])
        if i == 0 :
            print(layer_grads)
        #total_grads.append(np.mean(np.array(layer_grads)))

    return total_grads

In [23]:
from Models.SRResNet import SRResNet
from torch.utils.data import DataLoader
from Data.medmnist_loader import MedMNISTDataset
from Train.train_one_epoch import eval_epoch_save

# Try scale x 4 on model for x 8 and x 4
# load data and create test loader
first = 56
scale_first = int(224 / 56)
test_dataset1 = MedMNISTDataset(mode='test', low_size=first)
test_loader1 = DataLoader(test_dataset1, batch_size=1, shuffle=False)
# establish models
model1 = SRResNet(scaling_factor=4)
model2 = SRResNet(scaling_factor=4)
device = 'cuda'
wts1 = torch.load('/home/zoe/GhassanGT Dropbox/Zoe Fowler/Zoe/InSync/PhDResearch/Code/Federated-SuperResolution/Saved/pneumonia/best_model_4_resnet.pth')
wts2 = torch.load('/home/zoe/GhassanGT Dropbox/Zoe Fowler/Zoe/InSync/PhDResearch/Code/Federated-SuperResolution/Saved/pneumonia/best_model_8_resnet.pth')
model1.load_state_dict(wts1, strict=False)
model2.load_state_dict(wts2, strict=False)
model1 = model1.to(device)
model2 = model2.to(device)
criterion = nn.L1Loss().to(device)

print('Evaluating in distribution')
_, acc, outputs_indistrib = eval_epoch_save(data_loader=test_loader1, model=model1, criterion=criterion, device=device)
print('test psnr: ', acc)
# print('Evaluating out of distribution')
# _, acc, outputs_outdistrib = eval_epoch_save(data_loader=test_loader1, model=model2, criterion=criterion, device=device)
# print('test psnr: ', acc)

Using downloaded and verified file: /media/zoe/ssd/pneumoniamnist_224.npz
Evaluating in distribution
test psnr:  tensor(34.3951, device='cuda:0')


In [24]:
from Data.loader_from_array import MedMNIST_Testing
# form new dataset and loader of outputs
new_dataset1 = MedMNIST_Testing(input_arr = outputs_indistrib)
new_loader_test1 = DataLoader(new_dataset1, batch_size=1, shuffle=False)
ll = get_gradients(data_loader=new_loader_test1, model=model, criterion=criterion, device=device)

{'features.denseblock1': [0.2319102, 0.05846045, 1.4050945, 0.19895133, 0.09362672, 5.016035, 0.2932584, 0.09562707, 2.1293051, 0.2129423, 0.15873975, 5.649084, 0.2528194, 0.114099935, 2.0174346, 0.2111632, 0.14060876, 5.3234015, 0.21310095, 0.11778922, 1.8923578, 0.17163879, 0.12833926, 4.529271, 0.18748188, 0.1274887, 1.9307299, 0.19346951, 0.15858296, 4.092855, 0.14934824, 0.08542499, 1.5332464, 0.11596509, 0.12112134, 3.307009], 'features.denseblock2': [0.21627392, 0.1918361, 1.9442198, 0.21438509, 0.17882052, 5.296839, 0.24150787, 0.16844031, 2.1862109, 0.2038738, 0.15605606, 5.031063, 0.18203834, 0.14650594, 2.042695, 0.19059645, 0.15926647, 4.2690997, 0.16261187, 0.13049312, 1.870927, 0.1537934, 0.1280824, 3.479949, 0.13738832, 0.11322036, 1.6889151, 0.11090076, 0.106667675, 2.8229852, 0.11785458, 0.09718376, 1.3761982, 0.09315555, 0.09151158, 2.241416, 0.11595513, 0.10348609, 1.4749779, 0.09988447, 0.09447622, 2.401064, 0.097224005, 0.078973286, 1.3030804, 0.083657995, 0.082376

KeyboardInterrupt: 

In [13]:
np.mean(np.array(ll))

0.44923592

In [25]:
print('Evaluating out of distribution')
_, acc, outputs_outdistrib = eval_epoch_save(data_loader=test_loader1, model=model2, criterion=criterion, device=device)
print('test psnr: ', acc)
new_dataset2 = MedMNIST_Testing(input_arr = outputs_outdistrib)
new_loader_test2 = DataLoader(new_dataset2, batch_size=1, shuffle=False)
ll2 = get_gradients(data_loader=new_loader_test2, model=model, criterion=criterion, device=device)

Evaluating out of distribution
test psnr:  tensor(7.0845, device='cuda:0')
{'features.denseblock1': [0.2319102, 0.05846045, 1.4050945, 0.19895133, 0.09362672, 5.016035, 0.2932584, 0.09562707, 2.1293051, 0.2129423, 0.15873975, 5.649084, 0.2528194, 0.114099935, 2.0174346, 0.2111632, 0.14060876, 5.3234015, 0.21310095, 0.11778922, 1.8923578, 0.17163879, 0.12833926, 4.529271, 0.18748188, 0.1274887, 1.9307299, 0.19346951, 0.15858296, 4.092855, 0.14934824, 0.08542499, 1.5332464, 0.11596509, 0.12112134, 3.307009], 'features.denseblock2': [0.21627392, 0.1918361, 1.9442198, 0.21438509, 0.17882052, 5.296839, 0.24150787, 0.16844031, 2.1862109, 0.2038738, 0.15605606, 5.031063, 0.18203834, 0.14650594, 2.042695, 0.19059645, 0.15926647, 4.2690997, 0.16261187, 0.13049312, 1.870927, 0.1537934, 0.1280824, 3.479949, 0.13738832, 0.11322036, 1.6889151, 0.11090076, 0.106667675, 2.8229852, 0.11785458, 0.09718376, 1.3761982, 0.09315555, 0.09151158, 2.241416, 0.11595513, 0.10348609, 1.4749779, 0.09988447, 0.094

KeyboardInterrupt: 

In [18]:
ll2

[0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,
 0.44923592,